In [ ]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install ninja
!pip install gradio

In [ ]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

In [ ]:
model_name = "telord/Inst_Post_Generaton_Fine_tuned_DeciLM_7B"
# model_name = 'Deci/DeciLM-7B'

gpu_memory = torch.cuda.get_device_properties(0).total_memory

do_quantization = gpu_memory < 20e9

if do_quantization:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        # bfloat works only on A100 (or ampere supported chip)
        bnb_4bit_compute_dtype=torch.bfloat16,
        # if you're using a T4 or non-ampere chip comment out the above and run this instead:
        # bnb_4bit_compute_dtype=torch.float16
    )

    decilm = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        use_cache=True,
        trust_remote_code=True
    )
else:
    decilm = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        use_cache=True,
        trust_remote_code=True
    )


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

configuration_decilm.py:   0%|          | 0.00/576 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/113M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
deci_generator = pipeline("text-generation",
                          model=decilm,
                          tokenizer=tokenizer,
                          temperature=0.1,
                          device_map="auto",
                          # max_length=4096,
                          max_new_tokens = 100,

                          return_full_text=False)

system_prompt = "You are an AI assistant that generates instagram post if needed.Help as much as you can."

user_prompt = 'create a new post about the "Adventure" 25-liter backpack for $200, which is great for climbers'

In [ ]:
prompt = tokenizer.apply_chat_template([
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
], tokenize=False, add_generation_prompt=True)

response = deci_generator(prompt)[0]['generated_text']
print(prompt + response)

### System:
You are an AI assistant that generates instagram post if needed.Help as much as you can.
### User:
create a new post about the "Adventure" 25-liter backpack for $200, which is great for climbers
### Assistant:
Head out on your next adventure with the Adventure 25L backpack, designed to keep your gear organized and comfortable. With a durable, weather-resistant fabric and a comfortable suspension system, this pack is perfect for climbers and hikers alike.

#OspreyPacks | Featuring the Adventure 25L | #OspreyPacks #OspreyPacksAmbassador #OspreyPacksAdventure #Osp


In [ ]:
def generate_post(prompt):

    prompt = tokenizer.apply_chat_template([
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": prompt},
    ], tokenize=False, add_generation_prompt=True)

    response = deci_generator(prompt)[0]['generated_text']

    return response


iface = gr.Interface(
    fn=generate_post,
    inputs=gr.Textbox(lines=2, placeholder="Enter a prompt..."),
    outputs=gr.Textbox(),
    title="Instagram Post Generator"
)


iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://79bf145f02223d52b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
